Welcome, to Punisher. After running your strategy, view the live results with Dash

```
$ python -m punisher.charts.dash_charts.dash_record --name default
```

## Define Strategy

In [ ]:
%matplotlib inline
from punisher.common import *

from punisher.strategies.strategy import Strategy

class SimpleStrategy(Strategy):
    def __init__(self):
        super().__init__()

    def log_all(self, orders, data, ctx, time_utc):
        self.logger = ctx.logger
        if self.logger is not None:
            self.log_epoch_time(time_utc)
            self.log_ohlcv(data)
            self.log_orders(orders)
            self.log_performance(ctx)
            self.log_balance(ctx)
            self.log_positions(ctx)
            self.log_metrics(ctx)

    def handle_data(self, data, ctx):
        orders = []
        asset = Asset(c.XRP, c.BTC)
        price = data['close']
        quantity = 1.0

        if random.random() > 0.5:
            order = order_manager.build_limit_buy_order(
                ctx.exchange, asset, price, quantity)
        else:
            order = order_manager.build_limit_sell_order(
                ctx.exchange, asset, price, quantity)

        orders.append(order)

        # Add Metrics and OHLCV to Record
        self.update_metric('SMA', 5.0, ctx)
        self.update_metric('RSI', 10.0, ctx)
        self.update_ohlcv(data, ctx)

        self.log_all(orders, data, ctx, data['time_utc'])
        return orders

In [ ]:
strategy = SimpleStrategy()

## Backtest (Default config)

In [ ]:
runner.backtest('jupyter-demo-backtest', strategy)

## Backtest (Using Context)

In [ ]:
from punisher.trading.context import default_config, Context, TradingMode

config = default_config(TradingMode.BACKTEST)
context = Context.from_config(config)
runner.backtest('jupyter-demo-backtest', strategy, context=context)

## Simulate Orders (default config)

In [ ]:
runner.simulation('jupyter-demo-sim', strategy)

## Live Trade

In [ ]:
from datetime import datetime, timedelta
base = c.XRP
quote = c.BTC
asset = Asset(base, quote)
period = Timeframe.ONE_MIN
exchange_id = c.BINANCE

config = {
        'experiment': 'default-live-demo',
        'cash_asset': c.BTC,
        'starting_cash': 0.0,
        'store': FILE_STORE,
        'balance': Balance(starting_cash=0.0).to_dict(),
        'feed': {
            'fpath': ohlcv.get_price_data_fpath(asset, exchange_id, period.value['id']),
            'start': datetime.utcnow().strftime("%Y-%m-%dT%H:%M:%S"),
            'symbols': ['XRP/BTC'],
            'timeframe': Timeframe.ONE_MIN.name,
        },
        'exchange': {
            "exchange_id": exchange_id
        }
    }

config['feed']['name'] = EXCHANGE_FEED

 # must define your own config to live trade - so you know how it works :)
runner.live('jupyter-demo-live', strategy, config)